In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


# **READY THE MODEL AND TOKENIZER**

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    #model_name = "unsloth/Llama-3.2-3B-Instruct",
    # model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    model_name = "hyadess/UAP-EEE-llama-3.1-8b-16_bit_merged",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [5]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

# **FEW SHOT PROMPT CREATE FOR EVAL**

In [6]:
def create_5_shot_prompt(df, index):
    prompt = ""
    prompt += "### Examples:\n\n"
    row = df.iloc[index]
    prompt += f"#Example-1: \nQuery: {row['example_q_1']}\nResponse: {row['example_a_1']}\n"
    prompt += f"#Example-2: \nQuery: {row['example_q_2']}\nResponse: {row['example_a_2']}\n"
    prompt += f"#Example-3: \nQuery: {row['example_q_3']}\nResponse: {row['example_a_3']}\n"
    prompt += f"#Example-4: \nQuery: {row['example_q_4']}\nResponse: {row['example_a_4']}\n"
    prompt += f"#Example-5: \nQuery: {row['example_q_5']}\nResponse: {row['example_a_5']}\n\n\n"

    prompt += "### Conversation is given below:\n\n"

    return prompt

def create_3_shot_prompt(df, index):
    prompt = ""
    prompt += "## 3 Examples are given below:\n\n\n"
    row = df.iloc[index]
    prompt += f"#Example-1: \nQuestion: {row['example_q_1']}\nAnswer: {row['example_a_1']}\n\n"
    prompt += f"#Example-2: \nQuestion: {row['example_q_2']}\nAnswer: {row['example_a_2']}\n\n"
    prompt += f"#Example-3: \nQuestion: {row['example_q_3']}\nAnswer: {row['example_a_3']}\n\n"
    #prompt += f"###Example-4: \nQuestion: {row['example_q_4']}\nAnswer: {row['example_a_4']}\n\n"
    #prompt += f"###Example-5: \nQuestion: {row['example_q_5']}\nAnswer: {row['example_a_5']}\n\n"

    prompt += "## Conversation is given below:\n\n\n"

    return prompt


def create_1_shot_prompt(df, index):
    prompt = ""
    prompt += "## 1 Example is given below:\n\n\n"
    row = df.iloc[index]
    prompt += f"#Example-1: \nQuestion: {row['example_q_1']}\nAnswer: {row['example_a_1']}\n\n"
    #prompt += f"#Example-2: \nQuestion: {row['example_q_2']}\nAnswer: {row['example_a_2']}\n\n"
    #prompt += f"#Example-3: \nQuestion: {row['example_q_3']}\nAnswer: {row['example_a_3']}\n\n"
    #prompt += f"###Example-4: \nQuestion: {row['example_q_4']}\nAnswer: {row['example_a_4']}\n\n"
    #prompt += f"###Example-5: \nQuestion: {row['example_q_5']}\nAnswer: {row['example_a_5']}\n\n"

    prompt += "## Conversation is given below:\n\n\n"

    return prompt

# **MESSAGE ARRAY CREATION FROM TEST DATA**

In [7]:
import ast
def extract_fields(list_of_dicts):
    if isinstance(list_of_dicts, str):
        list_of_dicts = ast.literal_eval(list_of_dicts)

    from_values = [d['from'] for d in list_of_dicts]
    value_values = [d['value'] for d in list_of_dicts]
    return pd.Series([from_values, value_values])


# iteration every row and

def create_message_array(df,index):
    series=extract_fields(df['conversations'][index])
    # create a message list
    messages = []
    for i in range(len(series[0])):
        messages.append({"from": series[0][i], "value": series[1][i]})

    return messages


# **DATA PREP FOR HUMAN EVAL**

In [8]:
import pandas as pd

In [28]:
examples_df=pd.read_csv("/content/llama3.1_8b_finetuned_question4.csv")
# test_df=pd.read_csv("/content/test_data_shareGPT.csv")

In [10]:
# top 15 rows from test data
# test_df=test_df.head(24)
# test_df


In [ ]:
examples_df=examples_df.head(30)
examples_df

# **INFERENCE**

In [22]:
import re

In [23]:
def extract_last_section(text):
  pattern = r"<\|end_header_id\|>(.*?)<\|eot_id\|>"
  matches = re.findall(pattern, text, re.DOTALL)
  last_section = matches[-1].strip() if matches else ""
  return last_section



In [ ]:
from datasets import load_dataset

# Mount Google Dri
from google.colab import drive
drive.mount('/content/drive')

In [30]:
def generate_response(messages):
  inputs = tokenizer.apply_chat_template(
      messages,
      tokenize = True,
      add_generation_prompt = True, # Must add for generation
      return_tensors = "pt",
  ).to("cuda")

  outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True)
  text=tokenizer.batch_decode(outputs)[0]
  text = extract_last_section(text)
  return text

In [ ]:
zero_shot_results = []
one_shot_results = []
three_shot_results = []
five_shot_results = []
results=[]

for index, row in examples_df.iterrows():
    # if index != 26:
    #   continue

    # # for the automatic evaluation inference============================================================================================================================================================================
    # # for every 4 rows in test_df, there is one example row, so, for the first 4 rows in test_df, we use example 1, for 2nd 4 rows, we use example 2 and so on

    # one_shot_prompt=create_1_shot_prompt(examples_df,index//4)
    # three_shot_prompt=create_3_shot_prompt(examples_df,index//4)
    # five_shot_prompt=create_5_shot_prompt(examples_df,index//4)

    # # print the question1 column from the examples_df at index/4

    # print(f"===============================================================================similar question for few shot ait index {index}=========================================")
    # print(examples_df['question1'][index//4])




    # zero_shot_messages=[
    #     {"from": "system", "value": "provide a response to the latest query of the given conversation without repeating the query inself. you will be penalized for verbose response"}
    # ]
    # one_shot_messages=[
    #     {"from": "system", "value": "provide a response to the latest query of the given conversation without repeating the query inself. Use examples as reference to structure the response. you will be penalized for verbose response"+one_shot_prompt}
    # ]
    # three_shot_messages=[
    #     {"from": "system", "value": "provide a response to the latest query of the given conversation without repeating the query inself. Use examples as reference to structure the response. you will be penalized for verbose response"+three_shot_prompt}
    # ]
    # five_shot_messages=[
    #     {"from": "system", "value": "provide a response to the latest query of the given conversation without repeating the query inself. Use examples as reference to structure the response. you will be penalized for verbose response"+five_shot_prompt}
    # ]

    # message_array=create_message_array(test_df,index)

    # zero_shot_messages.extend(message_array)
    # one_shot_messages.extend(message_array)
    # three_shot_messages.extend(message_array)
    # five_shot_messages.extend(message_array)

    # zero_shot_response=generate_response(zero_shot_messages)
    # one_shot_response=generate_response(one_shot_messages)
    # three_shot_response=generate_response(three_shot_messages)
    # five_shot_response=generate_response(five_shot_messages)


    # zero_shot_results.append(zero_shot_response)
    # one_shot_results.append(one_shot_response)
    # three_shot_results.append(three_shot_response)
    # five_shot_results.append(five_shot_response)


    # print(f"======================================================zero shot for conversation {index}=====================================================================")
    # print(zero_shot_response)
    # print(f"======================================================one shot for conversation {index}=====================================================================")
    # print(one_shot_response)
    # print(f"======================================================three shot for conversation {index}=====================================================================")
    # print(three_shot_response)
    # print(f"======================================================five shot for conversation {index}=====================================================================")
    # print(five_shot_response)






    # for human evaluation inference==================================================================================================================================================================================

    few_shot_prompt="### Instruction:\n provide a short,and precise response for the latest query as if you are a university teacher.Follow the response format provided in the following example query-response pairs precisely.\n\n\n"+create_3_shot_prompt(examples_df,index)
    zero_shot_prompt="### Instruction:\n provide a short,and precise response for the latest query as if you are a university teacher.\n\n"

    messages=[
        {"from": "system", "value": zero_shot_prompt},
        {"from": "human", "value": row["question1"]},
        {"from": "gpt", "value": row["answer1"]},
        {"from": "human", "value": row["question2"]},
        {"from": "gpt", "value": row["answer2"]},
        {"from": "human", "value": row["question3"]},
        {"from": "gpt", "value": row["answer3"]},
        {"from": "human", "value": row["question4"]},
    ]

    result=generate_response(messages)
    results.append(result)
    print(f"======================================================human evaluation for conversation {index}=====================================================================")
    print(result)



# Add results to the DataFrame


examples_df['answer4'] = results

# save everything in test_df
# test_df['zero_shot_responses'] = zero_shot_results
# test_df['one_shot_responses'] = one_shot_results
# test_df['three_shot_responses'] = three_shot_results
# test_df['five_shot_responses'] = five_shot_results


examples_df.to_csv("/content/drive/MyDrive/llama3.1_8b_finetuned.csv", index=False)


print("Completions generated and saved")